# Recommend Products using SparkML

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

As for the notebooks, to run these you will need to register for a free Databricks
[Community Edition account](https://community.cloud.databricks.com/)

## import modules

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark.sql.functions as F

from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

## Design Schema

In [5]:
rate_schema = "`userid` string, `accoid` string, `rating` INT"
accos_schema = "`id` string, `title` string, `location` string, `price` int, `rooms` int, `rating` float, `type` string"
recommend_schema = "`userid` string, `accoid` string, `prediction` string"

## Read csv files to Spark DataFrame

In [7]:
rating_file_location = '/FileStore/tables/rating.csv'
dfRates = spark.read.csv(rating_file_location, rate_schema)

print(dfRates.count())
display(dfRates.take(10))

userid,accoid,rating
10,1,1
18,1,2
13,1,1
7,2,2
4,2,2
13,2,3
19,2,2
12,2,1
11,2,1
1,2,2


In [8]:
print(dfRates.count())

1187

In [9]:
accos_file_location = "/FileStore/tables/accommodation.csv"
dfAccos = spark.read.csv(accos_file_location, accos_schema)

print(dfAccos.count())
display(dfAccos.take(10))

id,title,location,price,rooms,rating,type
1,Comfy Quiet Chalet,Vancouver,50,3,3.0999999046325684,cottage
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage
3,Agreable Calm Place,London,65,4,4.800000190734863,house
4,Colossal Quiet Chateau,Paris,3400,16,2.700000047683716,castle
5,Homy Quiet Shack,Paris,50,1,1.100000023841858,cottage
6,Pleasant Quiet Place,Dublin,35,5,4.300000190734863,house
7,Vast Peaceful Fortress,Seattle,3200,24,1.899999976158142,castle
8,Giant Quiet Fortress,San Francisco,3400,12,4.099999904632568,castle
9,Giant Peaceful Palace,London,1500,20,3.5,castle
10,Sizable Calm Country House,Auckland,650,9,4.900000095367432,mansion


## Aggregations

In [11]:
df_agg = dfRates.agg(F.count('userid').alias('num_ratings'),
            F.countDistinct('userid').alias('distinct_users_rating'),
            F.max('rating').alias('best_rating'),
            F.min('rating').alias('worst_rating'),
            F.avg('rating').alias('avg_rating')
           )
display(df_agg)

num_ratings,distinct_users_rating,best_rating,worst_rating,avg_rating
1187,25,5,1,2.4667228306655433


## Left Join

In [13]:
df_leftjoined = dfAccos.withColumnRenamed('rating', 'ratings').join(dfRates, dfAccos.id == dfRates.accoid, how = 'left')
print(df_leftjoined.count())
display(df_leftjoined.take(10))

id,title,location,price,rooms,ratings,type,userid,accoid,rating
1,Comfy Quiet Chalet,Vancouver,50,3,3.0999999046325684,cottage,13,1,1
1,Comfy Quiet Chalet,Vancouver,50,3,3.0999999046325684,cottage,18,1,2
1,Comfy Quiet Chalet,Vancouver,50,3,3.0999999046325684,cottage,10,1,1
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,3,2,1
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,2,2,4
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,20,2,2
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,1,2,2
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,11,2,1
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,12,2,1
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,19,2,2


In [14]:
display(df_leftjoined.select('*').where(df_leftjoined.id == '100'))

id,title,location,price,rooms,ratings,type,userid,accoid,rating
100,Villa Staden,Stockholm,80000,4,4.2,house,null,null,null


In [15]:
display(df_leftjoined.select('*').where(df_leftjoined.id == '101'))

id,title,location,price,rooms,ratings,type,userid,accoid,rating


## Right Join

In [17]:
df_rightjoined = dfAccos.withColumnRenamed('rating', 'ratings').join(dfRates, dfAccos.id == dfRates.accoid, how = 'right')
print(df_rightjoined.count())
display(df_rightjoined.take(10))

id,title,location,price,rooms,ratings,type,userid,accoid,rating
1,Comfy Quiet Chalet,Vancouver,50,3,3.0999999046325684,cottage,10,1,1
1,Comfy Quiet Chalet,Vancouver,50,3,3.0999999046325684,cottage,18,1,2
1,Comfy Quiet Chalet,Vancouver,50,3,3.0999999046325684,cottage,13,1,1
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,7,2,2
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,4,2,2
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,13,2,3
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,19,2,2
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,12,2,1
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,11,2,1
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,1,2,2


In [18]:
display(df_rightjoined.select('*').where(df_rightjoined.id == '100'))

id,title,location,price,rooms,ratings,type,userid,accoid,rating


In [19]:
display(df_rightjoined.select('*').where(df_rightjoined.accoid == '101'))

id,title,location,price,rooms,ratings,type,userid,accoid,rating
null,null,null,null,null,null,null,23,101,5


## Inner Join

In [21]:
df_innerjoined = dfAccos.withColumnRenamed('rating' , 'ratings').join(dfRates, dfAccos.id == dfRates.accoid, how = 'inner')
print(df_innerjoined.count())
display(df_innerjoined.take(10))

id,title,location,price,rooms,ratings,type,userid,accoid,rating
1,Comfy Quiet Chalet,Vancouver,50,3,3.0999999046325684,cottage,10,1,1
1,Comfy Quiet Chalet,Vancouver,50,3,3.0999999046325684,cottage,18,1,2
1,Comfy Quiet Chalet,Vancouver,50,3,3.0999999046325684,cottage,13,1,1
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,7,2,2
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,4,2,2
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,13,2,3
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,19,2,2
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,12,2,1
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,11,2,1
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,1,2,2


In [22]:
display(df_innerjoined.select('*').where(df_innerjoined.id == '100'))

id,title,location,price,rooms,ratings,type,userid,accoid,rating


In [23]:
display(df_innerjoined.select('*').where(df_innerjoined.accoid == '101'))

id,title,location,price,rooms,ratings,type,userid,accoid,rating


## Train the model and recommend products with the model

### Train the model

In [26]:
model = ALS.train(dfRates.rdd, 20, 20)
print(type(model))

<class 'pyspark.mllib.recommendation.MatrixFactorizationModel'>

### Save the model

In [28]:
model.save(sc,'/FileStore/model_recommendations_v4.ml' )

### Use the trained model to predict what accommodations each user might be interested

In [30]:
allPredictions = None
for USER_ID in range(0, 100):
  dfUserRatings = dfRates.filter(dfRates.userid == USER_ID).rdd.map(lambda r: r.accoid).collect()
  rddPotential  = dfAccos.rdd.filter(lambda x: x[0] not in dfUserRatings)
  pairsPotential = rddPotential.map(lambda x: (USER_ID, x[0]))
  predictions = model.predictAll(pairsPotential).map(lambda p: (str(p[0]), str(p[1]), float(p[2])))
  predictions = predictions.takeOrdered(5, key=lambda x: -x[2]) # top 5
  print("predicted for user={0}".format(USER_ID))
  if (allPredictions == None):
    allPredictions = predictions
  else:
    allPredictions.extend(predictions)

predicted for user=0
predicted for user=1
predicted for user=2
predicted for user=3
predicted for user=4
predicted for user=5
predicted for user=6
predicted for user=7
predicted for user=8
predicted for user=9
predicted for user=10
predicted for user=11
predicted for user=12
predicted for user=13
predicted for user=14
predicted for user=15
predicted for user=16
predicted for user=17
predicted for user=18
predicted for user=19
predicted for user=20
predicted for user=21
predicted for user=22
predicted for user=23
predicted for user=24
predicted for user=25
predicted for user=26
predicted for user=27
predicted for user=28
predicted for user=29
predicted for user=30
predicted for user=31
predicted for user=32
predicted for user=33
predicted for user=34
predicted for user=35
predicted for user=36
predicted for user=37
predicted for user=38
predicted for user=39
predicted for user=40
predicted for user=41
predicted for user=42
predicted for user=43
predicted for user=44
predicted for user=45
predicted for user=46
predicted for user=47
predicted for user=48
predicted for user=49
predicted for user=50
predicted for user=51
predicted for user=52
predicted for user=53
predicted for user=54
predicted for user=55
predicted for user=56
predicted for user=57
predicted for user=58
predicted for user=59
predicted for user=60
predicted for user=61
predicted for user=62
predicted for user=63
predicted for user=64
predicted for user=65
predicted for user=66
predicted for user=67
predicted for user=68
predicted for user=69
predicted for user=70
predicted for user=71
predicted for user=72
predicted for user=73
predicted for user=74
predicted for user=75
predicted for user=76
predicted for user=77
predicted for user=78
predicted for user=79
predicted for user=80
predicted for user=81
predicted for user=82
predicted for user=83
predicted for user=84
predicted for user=85
predicted for user=86
predicted for user=87
predicted for user=88
predicted for user=89
predicted for user=90
predicted for user=91
predicted for user=92
predicted for user=93
predicted for user=94
predicted for user=95
predicted for user=96
predicted for user=97
predicted for user=98
predicted for user=99

### 5 Recommendations for every user

In [32]:
allPredictions

Out[170]: [('0', '75', 3.2174364246987786),
 ('0', '66', 3.214743761666351),
 ('0', '76', 3.2042630518688235),
 ('0', '39', 3.1912986564494874),
 ('0', '49', 3.128986838263849),
 ('1', '85', 2.950668383557024),
 ('1', '18', 2.856213031229358),
 ('1', '43', 1.8406984119756113),
 ('1', '46', 1.7087759360327675),
 ('1', '47', 1.5512846614631572),
 ('2', '39', 2.465133185395861),
 ('2', '66', 2.4094697911076763),
 ('2', '87', 2.3821459596786063),
 ('2', '61', 2.3513545394349897),
 ('2', '90', 2.2292788482252486),
 ('3', '6', 3.122784679983746),
 ('3', '51', 3.0308392109621227),
 ('3', '67', 2.9942345888128994),
 ('3', '43', 2.9400547945218927),
 ('3', '30', 2.8245411002837737),
 ('4', '7', 2.501831696398429),
 ('4', '65', 2.0720639596358987),
 ('4', '80', 1.8789917524414133),
 ('4', '69', 1.799330982678259),
 ('4', '73', 1.728678834726507),
 ('5', '98', 1.9069312100624645),
 ('5', '20', 1.7892340798275095),
 ('5', '39', 1.788419065762434),
 ('5', '87', 1.7128598403580533),
 ('5', '64', 1.7062678286192816),
 ('6', '38', 3.7805420158460685),
 ('6', '6', 3.7280897443225127),
 ('6', '30', 3.655049204816255),
 ('6', '49', 3.557090715506746),
 ('6', '99', 3.498187366194955),
 ('7', '34', 2.1802562123585085),
 ('7', '88', 1.7439370234780371),
 ('7', '58', 1.645807467312311),
 ('7', '44', 1.5402171641449292),
 ('7', '77', 1.516671037807611),
 ('8', '5', 3.0730215542015427),
 ('8', '47', 2.460342013506922),
 ('8', '81', 2.3789431956654696),
 ('8', '67', 2.305384187325073),
 ('8', '40', 2.2006025701372547),
 ('9', '75', 4.811241996678009),
 ('9', '66', 4.410211282574689),
 ('9', '99', 4.396793785191337),
 ('9', '59', 4.223070961041175),
 ('9', '49', 4.185195144938342),
 ('10', '46', 2.009643083378294),
 ('10', '74', 1.6374416743537963),
 ('10', '35', 1.5407037555882297),
 ('10', '40', 1.4737296499697337),
 ('10', '84', 1.4387841888860673),
 ('11', '21', 1.9498589004635343),
 ('11', '97', 1.9357201132976463),
 ('11', '78', 1.7863150567360147),
 ('11', '88', 1.7558136486918068),
 ('11', '26', 1.688182487491467),
 ('12', '66', 3.2987822387984886),
 ('12', '59', 3.2107818846808795),
 ('12', '99', 3.2003394199570794),
 ('12', '49', 3.1261279548486085),
 ('12', '39', 3.1163284907657474),
 ('13', '45', 3.114993521328958),
 ('13', '75', 3.0473312428120503),
 ('13', '86', 2.9152689336816895),
 ('13', '76', 2.827792691734399),
 ('13', '49', 2.7908943787944387),
 ('14', '97', 2.538503245114729),
 ('14', '45', 2.34295808980956),
 ('14', '73', 2.235258886436836),
 ('14', '23', 2.1793946718355217),
 ('14', '77', 2.1515455158383348),
 ('15', '45', 2.8517395388554667),
 ('15', '85', 2.3662901298486614),
 ('15', '73', 1.9945931839284976),
 ('15', '56', 1.6312355379580556),
 ('15', '87', 1.539464516902117),
 ('16', '80', 1.7949429422742031),
 ('16', '94', 1.345564740096311),
 ('16', '81', 1.3100486211200884),
 ('16', '35', 1.2441753239714144),
 ('16', '97', 1.1480990126157635),
 ('17', '87', 3.326671546548673),
 ('17', '25', 2.8148811200134185),
 ('17', '51', 2.5083012964336295),
 ('17', '95', 2.4778244422954683),
 ('17', '57', 2.03417672611633),
 ('18', '47', 2.216404563927998),
 ('18', '44', 2.207267067175839),
 ('18', '2', 2.1027732947575517),
 ('18', '67', 2.0557888533692625),
 ('18', '21', 1.9876052990452728),
 ('19', '30', 3.4359715350991586),
 ('19', '38', 3.4337270177121924),
 ('19', '59', 3.3287729200666822),
 ('19', '66', 3.2395476777423498),
 ('19', '99', 3.219790045376316),
 ('20', '75', 3.202872493256254),
 ('20', '76', 3.0856382204348405),
 ('20', '66', 2.999615042573135),
 ('20', '49', 2.903946113136035),
 ('20', '99', 2.888180921010441),
 ('21', '40', 1.7472261008826981),
 ('21', '18', 1.6841895570933205),
 ('21', '37', 1.6300450896171443),
 ('21', '77', 1.5839274010993174),
 ('21', '65', 1.5305974450238664),
 ('22', '5', 3.827939192669043),
 ('22', '80', 3.4952256690651415),
 ('22', '25', 3.216590558167859),
 ('22', '45', 3.02125074917034),
 ('22', '44', 2.9583618424147593),
 ('23', '7', 2.0722690457494455),
 ('23', '87', 1.6928930071097614)

### Prediction for user '1'

In [34]:
dfUserRatings = dfRates.filter(dfRates.userid == '1').rdd.map(lambda r: r.accoid).collect()
rddPotential  = dfAccos.rdd.filter(lambda x: x[0] not in dfUserRatings)
pairsPotential = rddPotential.map(lambda x: ('1', x[0]))

In [35]:
pairsPotential.collect()

Out[172]: [('1', '1'),
 ('1', '3'),
 ('1', '6'),
 ('1', '9'),
 ('1', '10'),
 ('1', '12'),
 ('1', '13'),
 ('1', '15'),
 ('1', '16'),
 ('1', '17'),
 ('1', '18'),
 ('1', '20'),
 ('1', '21'),
 ('1', '22'),
 ('1', '26'),
 ('1', '28'),
 ('1', '29'),
 ('1', '30'),
 ('1', '31'),
 ('1', '32'),
 ('1', '33'),
 ('1', '35'),
 ('1', '38'),
 ('1', '39'),
 ('1', '41'),
 ('1', '43'),
 ('1', '46'),
 ('1', '47'),
 ('1', '49'),
 ('1', '50'),
 ('1', '57'),
 ('1', '59'),
 ('1', '61'),
 ('1', '63'),
 ('1', '64'),
 ('1', '65'),
 ('1', '66'),
 ('1', '69'),
 ('1', '70'),
 ('1', '72'),
 ('1', '74'),
 ('1', '75'),
 ('1', '76'),
 ('1', '78'),
 ('1', '79'),
 ('1', '83'),
 ('1', '84'),
 ('1', '85'),
 ('1', '86'),
 ('1', '90'),
 ('1', '93'),
 ('1', '96'),
 ('1', '99'),
 ('1', '100')]

In [36]:
predictions = model.predictAll(pairsPotential).map(lambda p: (str(p[0]), str(p[1]), float(p[2])))

In [37]:
predictions_top5 = predictions.takeOrdered(5, key=lambda x: -x[2]) # top 5

In [38]:
predictions_top5

Out[175]: [('1', '85', 2.950668383557024),
 ('1', '18', 2.856213031229358),
 ('1', '43', 1.8406984119756113),
 ('1', '46', 1.7087759360327675),
 ('1', '47', 1.5512846614631572)]

### The top five accommodations are recommended to user '1'. Note that the quality of the recommendations are not great because the dataset was so small (the predicted ratings are not very high). Still, this lab illustrates the process you'd go through to create product recommendations.

In [40]:
predictions.sortBy(lambda kv: kv[2], False).collect()

Out[176]: [('1', '85', 2.950668383557024),
 ('1', '18', 2.856213031229358),
 ('1', '43', 1.8406984119756113),
 ('1', '46', 1.7087759360327675),
 ('1', '47', 1.5512846614631572),
 ('1', '70', 1.344146183310783),
 ('1', '64', 1.2951747391964636),
 ('1', '78', 1.28918496093518),
 ('1', '26', 1.2735783627786428),
 ('1', '29', 1.0638705933826627),
 ('1', '13', 1.0579473621893973),
 ('1', '57', 1.0489928456910897),
 ('1', '74', 1.0049710689931237),
 ('1', '99', 0.9278994265186369),
 ('1', '84', 0.9136169430177306),
 ('1', '69', 0.8688433966925021),
 ('1', '96', 0.8053521246921601),
 ('1', '79', 0.7803906619656822),
 ('1', '72', 0.6955727306938535),
 ('1', '49', 0.677583788158082),
 ('1', '83', 0.6554159697487717),
 ('1', '66', 0.6349306334487181),
 ('1', '15', 0.629170871425961),
 ('1', '10', 0.6121493719219631),
 ('1', '76', 0.5825906388957327),
 ('1', '38', 0.5815678781643958),
 ('1', '59', 0.5718594833953003),
 ('1', '63', 0.5614655149963219),
 ('1', '90', 0.5251661830938175),
 ('1', '65', 0.509211204046458),
 ('1', '61', 0.4567333193946882),
 ('1', '75', 0.44350822792480543),
 ('1', '16', 0.44146655335795226),
 ('1', '20', 0.4378518473275448),
 ('1', '41', 0.4125962135039727),
 ('1', '21', 0.4033795747407378),
 ('1', '28', 0.3464311531490085),
 ('1', '17', 0.33670776195612656),
 ('1', '32', 0.28015411579048993),
 ('1', '35', 0.2366509687476525),
 ('1', '39', 0.23219746617081874),
 ('1', '93', 0.19198000954363467),
 ('1', '31', 0.1618328027147412),
 ('1', '50', 0.14324898483752452),
 ('1', '1', 0.09822234633085532),
 ('1', '3', 0.0755100980405243),
 ('1', '22', 0.06087709331051382),
 ('1', '6', 0.05867712249662831),
 ('1', '9', -0.014974513207271221),
 ('1', '33', -0.034104274465554196),
 ('1', '86', -0.0753600765631455),
 ('1', '30', -0.23105425137894306),
 ('1', '12', -0.2606527766744975)]

### Create DataFram as dfRecommendation

In [42]:
dfRecommendation = spark.createDataFrame(allPredictions, ['userid', 'accoid', 'prediction'] )
display(dfRecommendation.take(10))

userid,accoid,prediction
0,75,3.2174364246987786
0,66,3.214743761666351
0,76,3.2042630518688235
0,39,3.1912986564494874
0,49,3.128986838263849
1,85,2.950668383557024
1,18,2.856213031229358
1,43,1.8406984119756113
1,46,1.7087759360327675
1,47,1.5512846614631572


### Find the recommendations for a specified user

#### dfRecommendation.accoid and dfAccos.id are duplicated in the joined DataFrame, could drop one of the columns

In [45]:
df_joined_rec_acco = dfRecommendation.join(dfAccos, dfRecommendation.accoid == dfAccos.id, how = 'inner').drop(dfAccos.id)
display(df_joined_rec_acco.take(10))

userid,accoid,prediction,title,location,price,rooms,rating,type
0,75,3.2174364246987786,Large Private Place,Berlin,50,4,3.5999999046325684,house
0,66,3.214743761666351,Beautiful Private Villa,London,80,2,2.4000000953674316,house
0,76,3.2042630518688235,Pleasant Calm Villa,Berlin,30,2,2.4000000953674316,house
0,39,3.1912986564494874,Beautiful Calm Villa,Vancouver,50,3,3.5,house
0,49,3.128986838263849,Big Private Villa,NYC,90,2,4.800000190734863,house
1,85,2.950668383557024,Nice Private Shack,Auckland,55,1,4.900000095367432,cottage
1,18,2.856213031229358,Big Peaceful Hut,Melbourne,60,2,2.4000000953674316,cottage
1,43,1.8406984119756113,Nice Private Hut,Melbourne,60,3,2.799999952316284,cottage
1,46,1.7087759360327675,Colossal Private Castle,San Francisco,1900,15,3.700000047683716,castle
1,47,1.5512846614631572,Sizable Calm Sately House,Seattle,900,10,1.5,mansion


In [46]:
display(df_joined_rec_acco.select('*').where(df_joined_rec_acco.userid == '10'))

userid,accoid,prediction,title,location,price,rooms,rating,type
10,46,2.009643083378294,Colossal Private Castle,San Francisco,1900,15,3.7,castle
10,74,1.6374416743537963,Giant Calm Fort,Melbourne,2400,12,2.3,castle
10,35,1.5407037555882297,Colossal Quiet Chateau,NYC,2300,14,4.6,castle
10,40,1.4737296499697337,Colossal Private Castle,Seattle,2900,24,1.5,castle
10,84,1.4387841888860673,Great Peaceful Sately House,Melbourne,700,8,3.2,mansion


### Save DataFrame dfRecommendation to csv formats

In [48]:
dfRecommendation.write.format('csv').option('header',True).mode('overwrite').option('sep',',').save('/FileStore/tables/recommend.csv')

### Load recommend.csv to DataFrame from FileStore

In [50]:
df_recommend = spark.read.csv('/FileStore/tables/test.csv', header = True)
display(df_recommend.take(10))

userid,accoid,prediction
21,40,1.7472261008826981
21,18,1.6841895570933205
21,37,1.6300450896171443
21,77,1.5839274010993174
21,65,1.5305974450238664
22,5,3.827939192669043
22,80,3.4952256690651415
22,25,3.216590558167859
22,45,3.02125074917034
22,44,2.9583618424147593


In [51]:
df_joined_rec_acco = dfRecommendation.join(dfAccos, dfRecommendation.accoid == dfAccos.id, how = 'inner').drop(dfAccos.id)
display(df_joined_rec_acco.select('*').where(df_joined_rec_acco.userid == '10'))

userid,accoid,prediction,title,location,price,rooms,rating,type
10,46,2.009643083378294,Colossal Private Castle,San Francisco,1900,15,3.7,castle
10,74,1.6374416743537963,Giant Calm Fort,Melbourne,2400,12,2.3,castle
10,35,1.5407037555882297,Colossal Quiet Chateau,NYC,2300,14,4.6,castle
10,40,1.4737296499697337,Colossal Private Castle,Seattle,2900,24,1.5,castle
10,84,1.4387841888860673,Great Peaceful Sately House,Melbourne,700,8,3.2,mansion
